# instalación de paquetes y verificación

In [1]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
from datetime import datetime
import string
from object_detection.utils import label_map_util
import glob
import os
import json

python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [2]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/tf1/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_featur

# Variables de entorno

In [26]:
training_dir = "training_masks"
images_dir  = "images_masks"
label_map = label_map_util.load_labelmap(training_dir+"/label_map.pbtxt")
label_map_dict = label_map_util.get_label_map_dict(label_map)
n_classes=len(label_map_dict.keys())
images_format=glob.glob(images_dir+"/*")[-1].split(".")[-1]
config_file_name="mask_rcnn_inception_v2_coco.config"
network_name="_".join(config_file_name.split("_")[0:2])
inference_graph_dir = "inference_graph_{}_{}_{}".format(network_name,datetime.now().day, datetime.now().month)


# creación del dataset

In [4]:
# borramos todos los pares de imágenes y anotaciones json donde no se haya detectado nada, ya que estos archivos sin
# detecciones producen un error en pasos posteriores
for json_path in glob.glob(images_dir+"/*.json"):
    with open(json_path) as json_file:
        data = json.load(json_file)
        if data["shapes"]==[]:
            print ("removing{} and {}".format(json_path,json_path.replace("json","JPG")))
            os.remove(json_path)
            os.remove(json_path.replace("json","JPG"))

In [5]:
# borramos las carpetas de train test y validación si es que existían de un entrenamiento anterior
!rm -rf {images_dir}/train {images_dir}/test {images_dir}/val

In [6]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test.
# el argumento -f indica el formato de las imágenes
!python3 scripts/train_test_split.py -i {images_dir} -o {images_dir} -tr 0.1 -vr 0.15 -f {images_format}
!tree -d {images_dir}

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images_masks
├── test
├── train
└── val

3 directories


In [7]:
# vamos a convertir estas fotos en y anotaciones en datasets de formato COCO. Tenemos que hacerlo para los sets de train
# test y validación. Tenemos que pasar como argumento archivo txt con los labels siguiendo el formato indiado en la
# documentación oficial de labelme https://github.com/wkentaro/labelme/tree/master/examples/instance_segmentation. Un ejemplo
# de cómo debe ser este archivo se muestra debajo
!python3 scripts/labelme2coco.py {images_dir}/train {images_dir}/train/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/test {images_dir}/test/coco --labels {training_dir}/labels.txt
!python3 scripts/labelme2coco.py {images_dir}/val {images_dir}/val/coco --labels {training_dir}/labels.txt
!tree -d {images_dir}

Creating dataset: images_masks/train/coco
Generating dataset from: images_masks/train/video2_frame9780.json
Generating dataset from: images_masks/train/video1_frame8950.json
Generating dataset from: images_masks/train/video2_frame8400.json
Generating dataset from: images_masks/train/video2_frame5100.json
Generating dataset from: images_masks/train/video2_frame2100.json
Generating dataset from: images_masks/train/video1_frame4500.json
Generating dataset from: images_masks/train/video2_frame1080.json
Generating dataset from: images_masks/train/video2_frame6780.json
Generating dataset from: images_masks/train/video1_frame9350.json
Generating dataset from: images_masks/train/video2_frame2580.json
Generating dataset from: images_masks/train/video2_frame8640.json
Generating dataset from: images_masks/train/video1_frame3000.json
Generating dataset from: images_masks/train/video1_frame8400.json
Generating dataset from: images_masks/train/video2_frame7020.json
Generating dataset from: images_ma

<img src="labels_txt.PNG">

In [8]:
# ahora borramos las imágenes y anotaciones de las carpetas train, test y val, ya que la carpeta coco dentro de cada 
# una de estas carpetas ya contiene las imágenes originales
!rm {images_dir}/test/*
!mv {images_dir}/test/coco/* {images_dir}/test
!rm -rf {images_dir}/test/coco 

!rm {images_dir}/train/*
!mv {images_dir}/train/coco/* {images_dir}/train
!rm -rf {images_dir}/train/coco 

!rm {images_dir}/val/*
!mv {images_dir}/val/coco/* {images_dir}/val
!rm -rf {images_dir}/val/coco 

!tree -d {images_dir}

rm: cannot remove 'images_masks/test/coco': Is a directory
rm: cannot remove 'images_masks/train/coco': Is a directory
rm: cannot remove 'images_masks/val/coco': Is a directory
images_masks
├── test
│   ├── JPEGImages
│   └── Visualization
├── train
│   ├── JPEGImages
│   └── Visualization
└── val
    ├── JPEGImages
    └── Visualization

9 directories


In [10]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 models/research/object_detection/dataset_tools/create_coco_tf_record.py --logtostderr \
--train_image_dir={images_dir}/train \
--val_image_dir={images_dir}/val \
--test_image_dir={images_dir}/test \
--train_annotations_file={images_dir}/train/annotations.json \
--val_annotations_file={images_dir}/val/annotations.json \
--testdev_annotations_file={images_dir}/test/annotations.json \
--output_dir={training_dir}/tfrecord \
--include_masks=True

I0218 10:29:36.263575 139680152872768 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0218 10:29:36.263960 139680152872768 create_coco_tf_record.py:412] 0 images are missing annotations.
I0218 10:29:36.264113 139680152872768 create_coco_tf_record.py:441] On image 0 of 241
I0218 10:29:50.924984 139680152872768 create_coco_tf_record.py:441] On image 100 of 241
I0218 10:30:08.305954 139680152872768 create_coco_tf_record.py:441] On image 200 of 241
I0218 10:30:15.042848 139680152872768 create_coco_tf_record.py:466] Finished writing, skipped 0 annotations.
I0218 10:30:15.457366 139680152872768 create_coco_tf_record.py:399] Found groundtruth annotations. Building annotations index.
I0218 10:30:15.457702 139680152872768 create_coco_tf_record.py:412] 0 images are missing annotations.
I0218 10:30:15.457803 139680152872768 create_coco_tf_record.py:441] On image 0 of 48
I0218 10:30:22.786206 139680152872768 create_coco_tf_record.py:466] Finished writing, 

In [18]:
!ls {training_dir}

label_map.pbtxt  mask_rcnn_inception_v2_coco.config  tfrecord
labels.txt	 temp.config


Esta celda se debe correr para crear un archivo de configuración temporal a partir de un archivo de configuración genérico incluído en los samples de la API de detección de objetos. Funcionará en muchos casos, pero es posible que en algunos casos sea necesario modificar este archivo a mano. Si se desean editar campos adicionales como el larning rate, el batch_size o la estructura de la red, se deberá modificar directamente el archivo de configuración mo editar la celda de debajo para añadir estas modificaciones

In [16]:
num_val_files= glob.glob(training_dir+"/tfrecord/coco_val.record*")[0].split(os.sep)[-1].split("-")[-1]
num_train_files= glob.glob(training_dir+"/tfrecord/coco_train.record*")[0].split(os.sep)[-1].split("-")[-1]
!sed 's#num_classes:.*#num_classes: {n_classes}#' {training_dir}/{config_file_name} >{training_dir}/temp.config
!sed -i 's#PATH_TO_BE_CONFIGURED#{training_dir}#' {training_dir}/temp.config
!sed -i 's#mscoco_label_map.pbtxt#label_map.pbtxt#' {training_dir}/temp.config
!sed -i 's#mscoco_val.*#tfrecord/coco_val.record-?????-of-{num_val_files}"#' {training_dir}/temp.config
!sed -i 's#mscoco_train.*#tfrecord/coco_train.record-?????-of-{num_train_files}"#' {training_dir}/temp.config

In [17]:
!cat {training_dir}/temp.config

# Mask R-CNN with Inception V2
# Configured for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "training_masks" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 4
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 800
        max_dimension: 1365
      }
    }
    number_of_stages: 3
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
     

# Entrenamiento y exportación del modelo

In [13]:
# borramos los archivos de un entrenamiento anterior si es que queremos empezar de nuevo en vez de reanudarlo
!rm {training_dir}/checkpoint\
{training_dir}/events*\
{training_dir}/graph.pbtxt\
{training_dir}/model*
!rm -r {training_dir}/eval_0


rm: cannot remove 'training_masks/eval_0': No such file or directory


In [19]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training_bb
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir={training_dir}/ \
--pipeline_config_path={training_dir}/temp.config \
--num_train_steps=1000000

W0218 10:33:10.442061 139942329120576 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 1000000
I0218 10:33:10.442294 139942329120576 config_util.py:552] Maybe overwriting train_steps: 1000000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0218 10:33:10.442344 139942329120576 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0218 10:33:10.442383 139942329120576 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0218 10:33:10.442425 139942329120576 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0218 10:33:10.442478 139942329120576 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, expor

# para correr el entrenamiento en la consola, copiar y pegar este comando

training_dir=training_bb
python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=$training_dir/ \
--pipeline_config_path=$training_dir/faster_rcnn_inception_v2_coco.config \
--num_train_steps=1000000

In [20]:
# corriendo esta celda sabremos cuál es el checkpoint más moderno
import glob
import os
max_step=0
for file in glob.glob(training_dir+"/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
        model_checkpoint_prefix="model.ckpt-{}".format(max_step)
print("En checkpoint más moderno es: \n{}/model.ckpt-".format(training_dir)+str(max_step))

En checkpoint más moderno es: 
training_masks/model.ckpt-2772


In [28]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido del modelo entrenado y  la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path {training_dir}/temp.config \
--trained_checkpoint_prefix {training_dir}/{model_checkpoint_prefix} \
--output_directory {inference_graph_dir}

Instructions for updating:
Please use `layer.__call__` method instead.
W0218 10:52:02.314922 140447977408320 deprecation.py:323] From /home/bigdata/anaconda3/envs/tf1/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0218 10:52:03.126022 140447977408320 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
I0218 10:52:03.216019 140447977408320 regularizers.py:99] Scale of 0 disables regularizer.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0218 10:52:03.216286 140447977408320 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0218 10:52:03.254338 140

In [30]:
# ahora creamos un archivo graph.pbtxt con la estructura concreta necesaria para usar este modelo con openCV 
!python3 scripts/tf_text_graph_faster_rcnn.py \
--input {inference_graph_dir}/frozen_inference_graph.pb \
--config {training_dir}/{config_file_name} \
--output {inference_graph_dir}/graph.pbtxt

Number of classes: 90
Scales:            [0.25, 0.5, 1.0, 2.0]
Aspect ratios:     [0.5, 1.0, 2.0]
Width stride:      16.000000
Height stride:     16.000000
Features stride:   16.000000
Instructions for updating:
Use tf.gfile.GFile.

2021-02-18 10:52:41.061927: I tensorflow/tools/graph_transforms/transform_graph.cc:317] Applying sort_by_execution_order



# Pediciendo en imágenes

In [ ]:
# debemos sustituir el parámetro -m por la carpeta donde está nuestro modelo exportado 
!python3 scripts/predict_on_images.py \
-i images_bb/test \
-o predictions_image_bb \
-m {inference_graph_dir} \
-l {training_dir}/label_map.pbtxt \
-f JPG